In [1]:
import time
import re
import pickle
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import string
import numpy as np

In [13]:
with open("srez.pickle", "rb") as f:
    rsrez = pd.DataFrame(pickle.load(f))

In [9]:
rsrez = rsrez[rsrez["Часть речи"].isin(["ПРИЛ", "НАР", "СУЩ", "", "ГЛАГ", "МЕСТ", "ПРЕД"]) & (~rsrez["Значения"].isin(["", "?"]))]

rsrez_pos = {
    'ПРИЛ': "adjective",
    'НАР': "adverb",
    'СУЩ': "noun",
    'ГЛАГ': "verb",
    'МЕСТ': "pronoun",
    '': None,
    'ПРЕД': "preposition"
}

rsrez = rsrez.assign(
    word=rsrez["Словарная статья"].str.lower(),
    definition=rsrez["Значения"].map(lambda x: re.sub(r"\n", " ", x)).map(lambda x: re.sub(r"[^\w\ ]", "", x).strip()),
    pos=srez["Часть речи"].map(srez_pos),
    sourse="sreznevskiy"
)[["old_rus_word", "definition", "pos", "sourse"]]

In [171]:
or_dict = rsrez[rsrez.definition.map(lambda x: " " not in x)]

In [172]:
or_w2w = rsrez[rsrez.definition.map(lambda x: " " in x)]

In [15]:
with open("parsed_410_tomes.pickle", "rb") as f:
    t410 = pickle.load(f)

In [82]:
t410 = t410[~t410.definition.astype(str).str.contains(r"^(?:к|то)\ ", case=False)]

In [98]:
t410 = t410[~t410.word.str.lower().isin(rsrez.old_rus_word.values)]

t410 = t410[~t410["info"].map(lambda x: "союз." in x if (x == x) else False)]

t410 = t410[t410.definition.notna()]

In [140]:
def ps_conv(l):
    if l != l:
        return None
    conv_dict = {
        "с.": "noun",
        "пр.": "adjective",
        "нар.": "adverb",
        "гл.": "verb",
        "мест.": "pronoun"
    } 
    for k, v in conv_dict.items():
        if k in l:
            return v
    return None

def clean_t410_definition(inp_s):
    s = inp_s.lower()
    s = re.sub(r"[^\w\ ]", "", s)
    s = " ".join([x for x in s.split() if all(y in "абвгдеёжзийклмнопрстуфхцчшщъыьэюя" for y in x)])
    return s

In [146]:
t410 = t410.assign(
    word=t410["word"].str.lower(),
    definition=t410["definition"].map(clean_t410_definition),
    pos=t410["info"].map(ps_conv),
    sourse="avanesov"
)[["word", "definition", "pos", "sourse"]]

In [284]:
# t410.head()

In [295]:
or_dict = or_dict.append(or_w2w, ignore_index=True)

In [181]:
or_dict = or_dict.append(t410[t410.translation.map(lambda x: " " not in x)], ignore_index=True)
or_dict = or_dict.assign(word=or_dict.word.str.replace(r"c", "с").map(lambda x: re.sub(r"(?:ѩ|˫а)", "я", x)))

In [297]:
leters = set()
for x in or_dict.word.values:
    leters.update(set(x))


In [300]:
# with open("or_dict.pickle", "wb") as f:
#     pickle.dump(or_dict, f)

In [8]:
with open("or_dict.pickle", "rb") as f:
    or_dict = pickle.load(f)

In [33]:
len(or_dict)

39329

In [2]:
with open("oed.pickle", "rb") as f:
    oed = pickle.load(f)

In [3]:
oed = oed[oed.word.map(lambda x: len(x) > 2)]

In [4]:
def pos_oed(s):
    if s == s:
        pos = {
            "sb.": "noun",
            "adj.": "adjective",
            "adv.": "adverb",
            "prep.": "preposition",
            "pp.": "preposition",
            "pron.": "pronoun",
            "v.": "verb",
        }
        for k in pos:
            if k in s:
                return pos[k]

oed = oed.assign(pos=oed.pos.map(pos_oed))

In [7]:
reg = re.compile(r"\b(?:t(?:he|o)|a)\ ")
oed = oed.assign(definition=oed.definition.map(lambda x: reg.sub("", x)))
reg = re.compile(r"\b\s?(?:a(?:s|nd|d[jv])|noun|verb|prep|conj|rel)\s?\b")
oed = oed.assign(definition=oed.definition.map(lambda x: reg.sub(" ", x)))

In [9]:
oed = oed.assign(source="mayhew_skeat")

In [24]:
# with open("oe_dict.pickle", "wb") as f:
#     pickle.dump(oe_dict, f)
# with open("oe_dict.pickle", "rb") as f:
#     oe_dict = pickle.load(f)

In [177]:
# letters = set()
# for word in oe_dict.word.values:
#     letters.update(set(word))
# letters

In [11]:
with open("oec.pickle", "rb") as f:
    oec = pd.DataFrame(pickle.load(f))

In [12]:
oec = oec[oec.word.map(lambda x: len(x) > 2) & (~oec.word.duplicated()) & oec.definition.notna()]

In [13]:
q = set("ʒ̣̄̆") - set("ʒ")
oec.assign(word=oec.word.map(lambda word: "".join(c for c in word if c not in q)))

reg = re.compile(r"(?:\([\w\-]+\)?|\(?[\w\-n̆]+\))")
oec = oec.assign(word=oec.word.map(lambda x: reg.sub("", x.lower())))

reg = "[^\-abcdefghijklmnopqrstuvwxyzæçéðüþāăčēěīōœūǎǒǔǧʒ]"
oec = oec[~oec.word.str.contains(reg)]


In [14]:
def pos_oec(s):
    if s == s:
        pos = {
            "n.": "noun",
            "ger.": "verb",
            "adj.": "adjective",
            "adv.": "adverb",
            "prep.": "preposition",
            "pp.": "preposition",
            "pron.": "pronoun",
            "v.": "verb",
        }
        for k in pos:
            if k in s:
                return pos[k]

oec = oec.assign(pos=oec.pos.map(pos_oec))

In [21]:
stopw = stopwords.words("english") + ["adj", "adv", "noun", "verb", "prep", "conj", "rel", "made", "used"]

In [22]:
def oec_tokenizer(s):
    res = s.lower().split("\n")[0]
    res = re.sub(r"(?:[\(].+?\)|[\[].+?[\]$]|[\(].+?$|[\[].+?$)", "", res)
    res = " ".join(word for word in re.findall(r"[abcdefghijklmnopqrstuvwxyz]+", res) if (len(word) > 2) and (word not in stopw))
    return res

oec = oec.assign(definition=oec.definition.map(oec_tokenizer))
# oec.assign(definition=oec.definition.map(oec_tokenizer))

In [31]:
"ȝ" == "ʒ"

False

In [29]:
ord("ȝ")

541

In [30]:
ord("ʒ")

658

In [24]:
oec = oec.assign(source="compendium")
oe_dict = oed.append(oec, ignore_index=True)
oe_dict = oe_dict.assign(definition=oe_dict.definition.str.strip())
oe_dict = oe_dict.assign(word=oe_dict.word.map(lambda x: x.translate({ord("ʒ"): "ȝ"})))

In [368]:
# with open("oe_dict_3.pickle", "wb") as f:
#     pickle.dump(oe_dict, f)

In [142]:
with open("doe.pickle", "rb") as f:
    doe = pd.DataFrame(pickle.load(f))

In [330]:
def doe_prep_word(s):
    res = re.sub(r"(?:ad[jv]|noun|pron|prep|and)", "", s.lower()).strip().split(", ")[0]
    res = res.translate({ord(x): None for x in "()*,.1234:?"})
    if len(res) < 3:
        return None
    else:
        return res

doe = doe.assign(word=doe.word.map(doe_prep_word))
doe = doe[doe.word.notna() & (~doe.word.str.contains(" "))]

In [349]:
def pos_doe(s):
    if s == s:
        pos = {
            "Noun": "noun",
            "Adj.": "adjective",
            "Adv.": "adverb",
            "prep.": "preposition",
            "Pronoun": "pronoun",
            "Vb.": "verb",
        }
        for k in pos:
            if k in s:
                return pos[k]

doe = doe.assign(pos=doe.pos.map(pos_doe))
# doe.assign(pos=doe.pos.map(pos_doe))

In [ ]:
# def doe_tokenizer(s):
#     res = s.lower().split("\n")[0]
#     res = re.sub(r"(?:[\(].+?\)|[\[].+?[\]$]|[\(].+?$|[\[].+?$)", "", res)
#     res = " ".join(word for word in re.findall(r"[abcdefghijklmnopqrstuvwxyz]+", res) if (len(word) > 2) and (word not in stopw))
#     return res

# oec = oec.assign(definition=oec.definition.map(oec_tokenizer))
# # oec.assign(definition=oec.definition.map(oec_tokenizer))

In [2]:
from pymystem3 import Mystem

def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            tagged.append(lemma.lower() + '_' + pos)
        except KeyError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    return tagged

In [3]:
import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]

print(mapping)

{'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ', 'APRO': 'DET', 'COM': 'ADJ', 'CONJ': 'SCONJ', 'INTJ': 'INTJ', 'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART', 'PR': 'ADP', 'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}


In [4]:
mapping["APRO"] = "ADJ"

In [19]:
def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
            else:
                tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
        except KeyError, IndexError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    return tagged

SyntaxError: invalid syntax (<ipython-input-19-e18ad2568cbf>, line 14)

In [6]:
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
rv_name = 'models/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz'
rusvec_model = gensim.models.KeyedVectors.load_word2vec_format(rv_name, binary=False)

In [9]:
or_dict.head()

,word,definition,pos,sourse
0,абидныи,обидный,adjective,sreznevskiy
1,аблань,яблоня,noun,sreznevskiy
2,абланьныи,яблоневый,adjective,sreznevskiy
3,аблъко,яблоко,noun,sreznevskiy
4,абрѣдиѥ,актиды,noun,sreznevskiy


In [10]:
pos_dict = {
    "adjective": "ADJ",
    "noun": "NOUN",
    "adverb": "ADV",
    "pronoun": "PRON",
    "verb": "VERB"
}


In [15]:
def single_word(row):
#     print(row)
    line = row["definition"]
    if " " not in line:
        return line
    pos = pos_dict.get(row.get("pos", None), None)
#     print(pos)
    stemmed = tag_mystem(text=line)
#     print(stemmed)
    imp_words = [word for word in stemmed if word in rusvec_model]
#     print(imp_words)
    if len(imp_words) == 0:
        return line.split()[0]
    single_vector = count_vector(imp_words)
    if pos is None:
        res = rusvec_model.similar_by_vector(single_vector, topn=1)
#         print(res)
        return res[0][0].split("_")[0]
    else:
        guesses = rusvec_model.similar_by_vector(single_vector, topn=3)
#         print(guesses)
#         print(pos)
        for guess, _ in guesses:
#             print(guess)
            if guess.endswith(pos):
                return guess.split("_")[0]
        else:
            return guesses[0][0].split("_")[0]
        
# def count_vector(imp_words):
#     vects = np.array([rusvec_model.wv[word] for word in imp_words])
#     single_vector = vects.mean(axis=0)
#     return single_vector

def count_vector(imp_words):
    vects = np.array([rusvec_model[word] for word in imp_words])
    if len(imp_words) == 1:
        return vects[0]
    noun_mask = [word.endswith("_NOUN") for word in imp_words]
    single_vector = np.average(vects, axis=0, weights=count_coefs(noun_mask))
    return single_vector

def count_coefs(mask):
    if not any(mask):
        mask[1] = True
    coefs = list()
    usual_coef = 1 / len(mask)
    for imp_mark in mask:
        if imp_mark:
            coef = usual_coef * 1.5
        else:
            coef = usual_coef
        coefs.append(coef)
    # softmax
    coefs = np.exp(coefs)/sum(np.exp(coefs))
    return coefs


In [16]:
single_word(or_dict.loc[20090])

'делать'

In [ ]:
or_dict = or_dict.assign(translation=or_dict.apply(single_word, axis=1))

In [ ]:
with open("or_dict_translated.pickle", "wb") as f:
    pickle.dump(or_dict, f)

In [146]:
test = or_dict.loc[20050:20100]